In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from tqdm.notebook import tqdm

In [ ]:
data = pd.read_csv('train.csv')
data['date'] = pd.to_datetime(data.date, dayfirst=False)

In [ ]:
data

In [ ]:
# Store item combinations
store_item_combis = np.unique(data[['store', 'item']].values, axis=0)

In [ ]:
n_lookback = 4 * 7 # Roughly one month
n_future = 7 # One week
all_sub_data = []
for combi in tqdm(store_item_combis):
    sub_data = data[(data.store == combi[0]) & (data.item == combi[1])].sort_values('date', ascending=True)
    # Past observation
    for i in range(1, n_lookback):
        sub_data['sales-{}'.format(i)] = sub_data['sales'].shift(i)
    # Unseen observation
    for i in range(1, n_future):
        sub_data['sales+{}'.format(i)] = sub_data['sales'].shift(-i)
    all_sub_data.append(sub_data)

In [ ]:
target_col_names = ['sales+{}'.format(i) for i in range(1, n_future)]
feature_col_names = ['sales-{}'.format(i) for i in range(1, n_lookback)]

In [ ]:
data = pd.concat(all_sub_data)
data.dropna(inplace=True)
data.head()

In [ ]:
y = data[target_col_names].values
x = data[feature_col_names].values
scaler = Normalizer()
scaler.fit(x)
x = scaler.transform(x)
y = scaler.transform(y)
print(x.shape, y.shape)

In [ ]:
np.save('x.npy', x)
np.save('y.npy', y)

In [ ]:
# date to categorical
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data.drop(columns=['date'], inplace=True)

In [ ]:
# cyclical date features
# normalization